# Noise Generation and Additive Noise Transform
This notebook illustrates some of the primary options for generating noise of various configurations using both the provided noise_generator utility and the additive noise transform

In [ ]:
import torchsig.transforms.functional as F
from torchsig.utils.dsp import (
    TorchSigComplexDataType,
    noise_generator
)

import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

Generate continous noise of colors white, pink, and red. Plot the magnitude spectra and compare the frequency roll-offs.

In [ ]:
# continous noise generator plots
N = 100000
freqs = np.fft.fftfreq(N)

# plots
plt.style.use('dark_background')
fig, ax = plt.subplots()

noise_white = noise_generator(N, 1.0, 'white', True, np.random.default_rng(42))
spectrum, freqs, _ = ax.magnitude_spectrum(noise_white, Fs=1.0, scale='linear', sides='twosided', color='white');

noise_pink = noise_generator(N, 1.0, 'pink', True, np.random.default_rng(42))
spectrum, freqs, _ = ax.magnitude_spectrum(noise_pink, Fs=1.0, scale='linear', sides='twosided', color='pink');

noise_red = noise_generator(N, 1.0, 'red', True, np.random.default_rng(42))
spectrum, freqs, _ = ax.magnitude_spectrum(noise_red, Fs=1.0, scale='linear', sides='twosided', color='red');

ax.set_yscale('log')
# ax.set_xscale('log')
ax.set_xscale('symlog', 
             base=10,          # Logarithm base
             linthresh=1/N)   # Threshold for linear region near zero

# Configure axis labels and grid
ax.set_xlabel('Frequency (Hz) [log]');
ax.set_ylabel('Magnitude [log]');
#plt.ylim([1e-5, None]);

Generate impulsive noise of colors white, pink, and red. Plot the magnitude spectra and compare the frequency roll-offs.

In [ ]:
N = 100000
freqs = np.fft.fftfreq(N)

# plots
plt.style.use('dark_background')
fig, ax = plt.subplots()

noise_white = noise_generator(N, 1.0, 'white', False, np.random.default_rng(42))
spectrum, freqs, _ = ax.magnitude_spectrum(noise_white, Fs=1.0, scale='linear', sides='twosided', color='white');

noise_pink = noise_generator(N, 1.0, 'pink', False, np.random.default_rng(42))
spectrum, freqs, _ = ax.magnitude_spectrum(noise_pink, Fs=1.0, scale='linear', sides='twosided', color='pink');

noise_red = noise_generator(N, 1.0, 'red', False, np.random.default_rng(42))
spectrum, freqs, _ = ax.magnitude_spectrum(noise_red, Fs=1.0, scale='linear', sides='twosided', color='red');

ax.set_yscale('log')
# ax.set_xscale('log')
ax.set_xscale('symlog', 
             base=10,          # Logarithm base
             linthresh=1/N)   # Threshold for linear region near zero

# Configure axis labels and grid
ax.set_xlabel('Frequency (Hz) [log]');
ax.set_ylabel('Magnitude [log]');
#plt.ylim([1e-5, None]);

Demonstrate the additive noise function transform.

In [ ]:
# continous noise generator plots
N = 100000
data_z = np.zeros((N,), dtype=TorchSigComplexDataType)
freqs = np.fft.fftfreq(N)

# plots
plt.style.use('dark_background')
fig = plt.figure()
ax1 = fig.add_subplot(2,1,1)
ax2 = fig.add_subplot(2,1,2)

noise_z = F.additive_noise(data_z, 1.0, 'white', True, np.random.default_rng(42))
spectrum, freqs, _ = ax.magnitude_spectrum(noise_z, Fs=1.0, scale='linear', sides='twosided', color='white');

ax1.plot(np.real(noise_z),label='Real')
ax1.plot(np.imag(noise_z),label='Imag')
ax1.legend(loc='upper right')

ax2.plot(freqs,spectrum)
ax2.set_yscale('log')
ax2.set_xscale('symlog', 
              base=10,          # Logarithm base
              linthresh=1/N)   # Threshold for linear region near zero
# # Configure axis labels and grid
ax2.set_xlabel('Frequency (Hz) [log]');
ax2.set_ylabel('Magnitude [log]');
plt.ylim([1e-5, None]);